# 테스트

In [32]:
from bs4 import BeautifulSoup
import pandas as pd

html = """
<html>
  <body>
    <div id="main" class="content">
      <h1>첫 번째 헤더</h1>
      <h1 class="header">두 번째 헤더</h1>
      <p class="description">설명입니다.</p>
      <a href="https://example1.com" class="link">링크 1</a>
      <a href="https://example2.com" class="link">링크 2</a>
      <a href="https://example3.com" class="external">링크 3</a>
    </div>
    <div id="main2" class="content">
      <div class="items">
        <div>
            <img class="thumbnail" src="/data/picture1.png" />
            <div id="title1" class="title">타이틀1</div>
            <div class="description">설명1</div>
        </div>
        <div>
            <img class="thumbnail" src="/data/picture2.png" />
            <div class="title">타이틀2</div>
            <div class="description">설명2</div>
        </div>
        <div>
            <img class="thumbnail" src="/data/picture3.png" />
            <div class="title test">타이틀3</div>
            <div class="description">설명3</div>
        </div>
      </div>
    </div>
  </body>
</html>
"""

# 특정하지 않으면 처음 한개만 추출된
soup = BeautifulSoup(html, 'html.parser')
soup.find('div', class_='title')

<div class="title" id="title1">타이틀1</div>

In [3]:
#첫번째 img 추출(둘 다 같음)
soup.find('img')
soup.find('img', src='/data/picture1.png')

<img class="thumbnail" src="/data/picture1.png"/>

In [ ]:
# class가 title인 div를 모두 추출
soup.find_all("div", class_="title")

[<div class="title">타이틀1</div>,
 <div class="title">타이틀2</div>,
 <div class="title test">타이틀3</div>]

In [15]:
# class가 items인 div 안의 div들을 모두 추출
# class가 items > div를 가져온다.
items = soup.find("div", class_="items")
div_list = items.find_all("div")
for div in div_list:
    print(div)
    print("---")

<div>
<img class="thumbnail" src="/data/picture1.png"/>
<div class="title">타이틀1</div>
<div class="description">설명1</div>
</div>
---
<div class="title">타이틀1</div>
---
<div class="description">설명1</div>
---
<div>
<img class="thumbnail" src="/data/picture2.png"/>
<div class="title">타이틀2</div>
<div class="description">설명2</div>
</div>
---
<div class="title">타이틀2</div>
---
<div class="description">설명2</div>
---
<div>
<img class="thumbnail" src="/data/picture3.png"/>
<div class="title test">타이틀3</div>
<div class="description">설명3</div>
</div>
---
<div class="title test">타이틀3</div>
---
<div class="description">설명3</div>
---


In [16]:
# img 태그에서 src를 추출
print(soup.find("img").attrs) # dict 형태로 추출
print(soup.find("img").attrs["src"])
print(soup.find("img").get("src")) # 에러가 발생하지 않음 => None 응답
print(soup.find("img")["src"])

{'class': ['thumbnail'], 'src': '/data/picture1.png'}
/data/picture1.png
/data/picture1.png
/data/picture1.png


In [28]:
items_html = soup.find('div', class_='items')

div_outputs = items_html.find_all("div", recursive=False)

print(div_outputs)
print("="*50)

data_list = []

for output in div_outputs:
    
    temp_dict = {}
    temp_dict["image"] = output.find("img", class_="thumbnail").attrs.get("src")
    temp_dict["title"] = output.find("div", class_="title").text
    temp_dict["description"] = output.find("div", class_="description").text
    
    data_list.append(temp_dict)
    
    print(temp_dict)
    print("="*50)
    
    data = pd.DataFrame(data_list)
data

[<div>
<img class="thumbnail" src="/data/picture1.png"/>
<div class="title">타이틀1</div>
<div class="description">설명1</div>
</div>, <div>
<img class="thumbnail" src="/data/picture2.png"/>
<div class="title">타이틀2</div>
<div class="description">설명2</div>
</div>, <div>
<img class="thumbnail" src="/data/picture3.png"/>
<div class="title test">타이틀3</div>
<div class="description">설명3</div>
</div>]
{'image': '/data/picture1.png', 'title': '타이틀1', 'description': '설명1'}
{'image': '/data/picture2.png', 'title': '타이틀2', 'description': '설명2'}
{'image': '/data/picture3.png', 'title': '타이틀3', 'description': '설명3'}


,image,title,description
0,/data/picture1.png,타이틀1,설명1
1,/data/picture2.png,타이틀2,설명2
2,/data/picture3.png,타이틀3,설명3


In [ ]:
soup.select_one('#title1') # 단일 선택

<div class="title" id="title1">타이틀1</div>

In [ ]:
soup.select('.title') # 복수 선택

[<div class="title" id="title1">타이틀1</div>,
 <div class="title">타이틀2</div>,
 <div class="title test">타이틀3</div>]

In [ ]:
soup.select('div.items > div:nth-child(3)') # 자식요소 선택

[<div>
 <img class="thumbnail" src="/data/picture3.png"/>
 <div class="title test">타이틀3</div>
 <div class="description">설명3</div>
 </div>]

# 교보문고 크롤링

In [ ]:
import requests
from bs4 import BeautifulSoup

base_url = 'https://search.kyobobook.co.kr/search'

params = {
    'keyword': '인공지능',
    'gbCode': 'TOT',
    'target': 'kyobo'
}

# 크롤링 차단 우회를 위한 header 추가(네트워크 탭 다운된 파일 정보 참조)
headers = {
    "User-Agent": 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36',
}

response = requests.get(base_url, params=params, headers=headers)

html = response.text
soup = BeautifulSoup(html, 'html.parser')

for title in soup.select('.prod_info > span[id]'):
    print(title.text)

print('='*100)

for title in soup.select('[id^="cmdtName_S"]'):
    print(title.text)    

예약판매
인공지능입문
예약판매
인공지능과 죽음
된다! 하루 만에 끝내는 챗GPT 활용법
AI 리터러시: 인공지능 필수 지식부터 완벽 활용까지
MCP로 똑똑하게 일하는 법
비전공자도 이해할 수 있는 AI 지식(10만부 기념 개정판)
김대식 교수의 어린이를 위한 인공지능 : 메타버스부터 챗GPT까지
혼자 공부하는 머신러닝+딥러닝
서울대학교 AI융합교육학과 인공지능 수업 가이드
Google로 칼퇴하는 교사들의 시크릿
7가지 프로젝트로 배우는 LLM AI 에이전트 개발
2025 AI 트렌드 100
개념과 원리가 보이는
우리학교 인공지능 수업 2
AI 2041
인공지능은 나의 읽기-쓰기를 어떻게 바꿀까
프롬프트 엔지니어링으로 인공지능 제대로 일 시키기
개념과 원리가 보이는
우리학교 인공지능 수업 1
인공지능 윤리를 부탁해
인공지능이 가르칠 수 있다는 착각
AI 전쟁
인공지능입문
인공지능과 죽음
된다! 하루 만에 끝내는 챗GPT 활용법
AI 리터러시: 인공지능 필수 지식부터 완벽 활용까지
MCP로 똑똑하게 일하는 법
비전공자도 이해할 수 있는 AI 지식(10만부 기념 개정판)
김대식 교수의 어린이를 위한 인공지능 : 메타버스부터 챗GPT까지
혼자 공부하는 머신러닝+딥러닝
서울대학교 AI융합교육학과 인공지능 수업 가이드
Google로 칼퇴하는 교사들의 시크릿
7가지 프로젝트로 배우는 LLM AI 에이전트 개발
2025 AI 트렌드 100
우리학교 인공지능 수업 2
AI 2041
인공지능은 나의 읽기-쓰기를 어떻게 바꿀까
프롬프트 엔지니어링으로 인공지능 제대로 일 시키기
우리학교 인공지능 수업 1
인공지능 윤리를 부탁해
인공지능이 가르칠 수 있다는 착각
AI 전쟁
